In [ ]:
"""
Help me write a blog post no longer than 1,000 words.

The Tool should be able to do 3 things:

-   research a topic on the web
-   extact specific information from documents I provide
-   write a blog post from the research and documents

"""

In [2]:
"""
    ConversationalAgent with three tools:
    -   WebSearch
    -   DocumentSearch
    -   BlogPostWriter

"""
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from haystack.nodes import WebRetriever, PromptTemplate, PromptNode
from haystack.pipelines import WebQAPipeline

# WebRetriever tool object
retriever = WebRetriever(
    api_key = os.getenv("SERPAPI_API_KEY"),
    search_engine_provider = "SerpAPI",
    mode  = "raw_documents"
)

# Prompt to instruct the LLM
prompt_template = PromptTemplate(
    prompt= """
    Synthesize a comprehensive answer from a web search for the given question.
    Provide a clear and concise response that summarizes the key points and information presented in the search.
    Your answer should be concise and be no longer than 500 words.
    \n\n Search results: {join(documents)} \n\n Question: {query} \n\n Answer:""",
)

# Insantiate the LLM with URI, API key, and prompt template
prompt_node = PromptNode(
    model_name_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    api_key = os.getenv("HF_TOKEN"),
    default_prompt_template = prompt_template,
    model_kwargs = {
        "max_new_tokens": 1000,
        "context_length": 2500 
    },
)

# Wrap the retriever and prompt node in a pipeline object
WebSearchTool = WebQAPipeline(
    retriever=retriever,
    prompt_node=prompt_node
)

In [ ]:
from haystack.nodes import PromptNode
from haystack.agents.conversational import ConversationalAgent
from haystack.agents.base import Tool

# Initialize a PromptNode and the desired tools
prompt_node = PromptNode("gpt-3.5-turbo",
                         api_key=os.getenv("OPENAI_API_KEY"),
                         max_length=512         
                    )

tools = [Tool(name="ExampleTool", pipeline_or_node=example_tool_node)]

# Create the ConversationalAgent instance
agent = ConversationalAgent(prompt_node, tools=tools)

# Use the agent in a chat application
while True:
    user_input = input("Human (type 'exit' or 'quit' to quit): ")
    if user_input.lower() == "exit" or user_input.lower() == "quit":
        break
    else:
        assistant_response = agent.run(user_input)
        print("Assistant:", assistant_response)

In [ ]:
"""
Help me dissect academic research papers.

The Tool should be able to do 4 things:

-   extract key phrases, sentences, and paragraphs from the research paper
-   highlight interesting exhibits, tables, and figures from the research paper
-   hihglight the key references and citations from the research paper
-   write a short twitter thread about the research paper using the extracted information

"""